# **Install required libraries**

In [1]:
!pip install opencv-python pillow chess tensorflow
!apt-get install -y stockfish


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 52.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147775 sha256=6b57119b8fe9c37a803ee4e2b0c3c250b5139fe1f89984bbb02159ce06ced28e
  Stored in directory: /root/.cache/pip/wheels/fb/5d/5c/59a62d8a695285e59ec9c1f66add6f8a9ac4152499a2be0113
Successfully built chess
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  polyglot xboard | scid
The following NEW packages will be installed:
  stockfish
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 24.8 MB of archives.
After this operation, 47.4 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 stockfish amd64 14.1-1 [24.8 MB]
Fetched 24.8 MB in 1s (21.6 MB/s)
Selecting previously unselected package stockfish.
(Reading database ... 126281 files and directories cur

# **Upload files image file , .h5 , .pkl**  


In [2]:
from google.colab import files

uploaded = files.upload()  # Upload .h5


Saving chess_piece_model.h5 to chess_piece_model.h5


In [3]:
from google.colab import files

uploaded = files.upload()  # Upload  .pkl


Saving label_encoder.pkl to label_encoder.pkl


In [4]:
from google.colab import files

uploaded = files.upload()  # Upload .h5, .pkl, and image file


Saving your_image.png to your_image.png


# **Load models**

In [6]:
from tensorflow.keras.models import load_model

model_h5 = load_model("chess_piece_model.h5")  # Replace with actual filename


In [9]:
import pickle

with open("label_encoder.pkl", "rb") as f:
    preprocessor = pickle.load(f)


# Load and Preprocess Image

In [10]:
import cv2
import numpy as np
from PIL import Image

def load_and_split_board(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (512, 512))  # Resize to 8x8 board
    squares = []

    for row in range(8):
        for col in range(8):
            square = img[row*64:(row+1)*64, col*64:(col+1)*64]
            square = cv2.cvtColor(square, cv2.COLOR_BGR2RGB)
            square = square / 255.0
            squares.append(square)

    return np.array(squares).reshape(-1, 64, 64, 3)


# **Predict Pieces Using Model**

In [11]:
classes = [
    'empty', 'white_pawn', 'white_rook', 'white_knight',
    'white_bishop', 'white_queen', 'white_king',
    'black_pawn', 'black_rook', 'black_knight',
    'black_bishop', 'black_queen', 'black_king'
]

def predict_board(image_path):
    board = load_and_split_board(image_path)
    predictions = model_h5.predict(board)
    predicted_classes = [classes[np.argmax(p)] for p in predictions]
    return np.array(predicted_classes).reshape(8, 8)


# **Convert Predicted Board to FEN**

In [12]:
def board_to_fen(predicted_board):
    fen = ""
    for row in predicted_board:
        empty_count = 0
        for square in row:
            if square == "empty":
                empty_count += 1
            else:
                if empty_count > 0:
                    fen += str(empty_count)
                    empty_count = 0

                piece_letter = square.split("_")[1][0]
                if "white" in square:
                    fen += piece_letter.upper()
                else:
                    fen += piece_letter.lower()
        if empty_count > 0:
            fen += str(empty_count)
        fen += "/"
    fen = fen[:-1]  # Remove last slash
    fen += " w KQkq - 0 1"  # Add standard end
    return fen


# **Run the Whole Pipeline**

In [13]:
image_path = "your_image.png"  # Replace with your image name

predicted_board = predict_board(image_path)
fen = board_to_fen(predicted_board)

print("FEN:", fen)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
FEN: KBKBKBKB/BBBBBRBR/KBKBKBKB/BKBKBKBK/kBBBpBKB/BKBpBBBK/KBKBKBpB/BKBKBKBK w KQkq - 0 1
